<a href="https://colab.research.google.com/github/WHU-Peter/COMP6248-Deep-Learning/blob/master/DL_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 15.0MB/s 


In [ ]:
import torch
import torchbearer
import torch.nn.functional as F
from torch import nn
import torchbearer
from torchbearer import Trial
from torch import optim
from sklearn import metrics
from torchvision.models import resnet50
from urllib.request import urlopen
import numpy as np
from sklearn.svm import SVC
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
from os.path import exists
if not exists('data'):
    !wget -O boat-data.zip https://artist-cloud.ecs.soton.ac.uk/index.php/s/eAhIkhhdxgmhRHj/download
    !unzip boat-data.zip

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
loss_function = nn.CrossEntropyLoss()
# the number of images that will be processed in a single step
batch_size=128
# the size of the images that we'll learn on - we'll shrink them from the original size for speed
image_size=(30, 100)

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor()  # convert to tensor
])

train_dataset = ImageFolder("data/train", transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = ImageFolder("data/valid", transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = ImageFolder("data/test", transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class BetterCNN(nn.Module):
    def __init__(self, n_channels_in, n_classes):
        super(BetterCNN, self).__init__()
        self.conv1 = nn.Conv2d(n_channels_in, 30, (5, 5), padding=0)
        self.conv2 = nn.Conv2d(30, 15, (3, 3), padding=0)
        self.fc1 = nn.Linear(1725, 128)
        self.fc2 = nn.Linear(128, 50)
        self.fc3 = nn.Linear(50, n_classes)
    
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        out = self.conv2(out)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        out = F.dropout(out, 0.2)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        return out

In [ ]:
model = BetterCNN(3, len(train_dataset.classes))
model.to(device)

# define the loss function and the optimiser
optimiser = optim.Adam(model.parameters())

trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(train_loader, val_generator=val_loader, test_generator=test_loader)
trial.run(epochs=30)

predictions = trial.predict()
predicted_classes = predictions.argmax(1).cpu()
true_classes = list(x for (_,x) in test_dataset.samples)
print(metrics.classification_report(true_classes, predicted_classes, target_names=train_dataset.classes))


                         precision    recall  f1-score   support

              Alilaguna       0.74      0.74      0.74        19
              Ambulanza       0.26      0.45      0.33        22
               Barchino       0.28      0.14      0.18        51
                Gondola       0.00      0.00      0.00         3
          Lanciafino10m       0.00      0.00      0.00         7
              Motobarca       0.33      0.25      0.29        59
Motopontonerettangolare       1.00      0.33      0.50         3
          MotoscafoACTV       0.50      1.00      0.67         1
               Mototopo       0.77      0.74      0.76       274
              Patanella       0.33      0.81      0.47        74
                Polizia       0.12      0.07      0.09        15
        Raccoltarifiuti       0.07      0.05      0.06        19
           Sandoloaremi       0.00      0.00      0.00         3
                   Topa       0.00      0.00      0.00        29
          VaporettoACTV

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model = resnet50(pretrained=True)
model.avgpool = nn.AdaptiveAvgPool2d((1,1))
model.fc = nn.Linear(2048, len(train_dataset.classes))
model.train()

for param in model.parameters():
    param.requires_grad = False
model.fc.weight.requires_grad = True #unfreeze last layer weights
model.fc.bias.requires_grad = True #unfreeze last layer biases
model.to(device)

optimiser = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4) #only optimse non-frozen layers
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(train_loader, val_generator=val_loader, test_generator=test_loader)
trial.run(epochs=30)
predictions = trial.predict()
predicted_classes = predictions.argmax(1).cpu()
true_classes = list(x for (_,x) in test_dataset.samples)
print(metrics.classification_report(true_classes, predicted_classes, target_names=train_dataset.classes))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



                         precision    recall  f1-score   support

              Alilaguna       0.38      0.26      0.31        19
              Ambulanza       0.64      0.41      0.50        22
               Barchino       0.54      0.14      0.22        51
                Gondola       0.00      0.00      0.00         3
          Lanciafino10m       0.00      0.00      0.00         7
              Motobarca       0.19      0.05      0.08        59
Motopontonerettangolare       0.00      0.00      0.00         3
          MotoscafoACTV       0.00      0.00      0.00         1
               Mototopo       0.60      0.89      0.72       274
              Patanella       0.38      0.42      0.40        74
                Polizia       0.00      0.00      0.00        15
        Raccoltarifiuti       0.64      0.47      0.55        19
           Sandoloaremi       0.00      0.00      0.00         3
                   Topa       0.00      0.00      0.00        29
          VaporettoACTV

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model = resnet50(pretrained=True)
model.avgpool = nn.AdaptiveAvgPool2d((1,1))
model.fc = nn.Linear(2048, len(train_dataset.classes))
model.train()

# Freeze layers by not tracking gradients
for param in model.parameters():
    param.requires_grad = False
for param in list(model.children())[-3].parameters():
    param.requires_grad = True
model.fc.weight.requires_grad = True #unfreeze last layer weights
model.fc.bias.requires_grad = True #unfreeze last layer biases
model.to(device)

optimiser = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(train_loader, val_generator=val_loader, test_generator=test_loader)
trial.run(epochs=30)
predictions = trial.predict()
predicted_classes = predictions.argmax(1).cpu()
true_classes = list(x for (_,x) in test_dataset.samples)
print(metrics.classification_report(true_classes, predicted_classes, target_names=train_dataset.classes))


                         precision    recall  f1-score   support

              Alilaguna       1.00      0.63      0.77        19
              Ambulanza       0.82      0.64      0.72        22
               Barchino       0.50      0.29      0.37        51
                Gondola       0.50      0.67      0.57         3
          Lanciafino10m       0.00      0.00      0.00         7
              Motobarca       0.30      0.22      0.25        59
Motopontonerettangolare       1.00      0.67      0.80         3
          MotoscafoACTV       1.00      1.00      1.00         1
               Mototopo       0.81      0.86      0.84       274
              Patanella       0.46      0.68      0.55        74
                Polizia       0.30      0.40      0.34        15
        Raccoltarifiuti       0.44      0.84      0.58        19
           Sandoloaremi       0.00      0.00      0.00         3
                   Topa       0.23      0.10      0.14        29
          VaporettoACTV

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
if not exists('training_features.npy'):
    !wget -O Resnet50Features.zip https://artist-cloud.ecs.soton.ac.uk/index.php/s/P68OB07DquOwSR7/download
    !unzip Resnet50Features.zip

training_features = np.load('training_features.npy')
training_labels = np.load('training_labels.npy')

valid_features = np.load('valid_features.npy')
valid_labels = np.load('valid_labels.npy')

testing_features = np.load('testing_features.npy')
testing_labels = np.load('testing_labels.npy')

clf = SVC()
clf.fit(training_features, training_labels)
predicted_classes = clf.predict(testing_features)
print(metrics.classification_report(testing_labels, predicted_classes, target_names=train_dataset.classes))

--2021-04-05 23:32:18--  https://artist-cloud.ecs.soton.ac.uk/index.php/s/P68OB07DquOwSR7/download
Resolving artist-cloud.ecs.soton.ac.uk (artist-cloud.ecs.soton.ac.uk)... 152.78.100.1, 2001:630:d0:f400::f001
Connecting to artist-cloud.ecs.soton.ac.uk (artist-cloud.ecs.soton.ac.uk)|152.78.100.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38384449 (37M) [application/zip]
Saving to: ‘Resnet50Features.zip’

Resnet50Features.zi 100%[===================>]  36.61M  23.0MB/s    in 1.6s    

2021-04-05 23:32:20 (23.0 MB/s) - ‘Resnet50Features.zip’ saved [38384449/38384449]

Archive:  Resnet50Features.zip
  inflating: training_features.npy   
  inflating: training_labels.npy     
  inflating: testing_labels.npy      
  inflating: valid_features.npy      
  inflating: valid_labels.npy        
  inflating: testing_features.npy    
                         precision    recall  f1-score   support

              Alilaguna       0.90      1.00      0.95        19
       

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
